In [5]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.11.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")


In [6]:
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

import java.time.LocalTime

// Location class representing a point in Geneva
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
}

// Skill enumeration
enum class Skill {
    FIRE_CERTIFICATION,
    BODYGUARD_CERTIFICATION,
    FIRST_AID,
    // Add more skills as needed
}

// Agent class representing a security agent
data class Agent(
    @PlanningId
    val id: Long,
    val name: String,
    val homeLocation: Location,
    val vehicleHomeLocation: Location,
    val skills: Set<Skill>,
    val weeklyWorkloadHours: Int = 40
) {
    // No-arg constructor required for Timefold
    constructor() : this(0, "", Location(0.0, 0.0), Location(0.0, 0.0), emptySet(), 40)
}

// Meeting class representing a client meeting
@PlanningEntity
data class Meeting(
    @PlanningId
    val id: Long,
    val clientName: String,
    val location: Location,
    val requiredSkills: Set<Skill>,
    val durationHours: Int,
    val timeWindowStart: LocalTime,
    val timeWindowEnd: LocalTime,
    val isNightMeeting: Boolean = false
) {
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"], nullable = true)
    var assignedAgent: Agent? = null

    // No-arg constructor required for Timefold
    constructor() : this(0, "", Location(0.0, 0.0), emptySet(), 1, LocalTime.MIN, LocalTime.MAX)
}

// Solution class representing the entire problem
@PlanningSolution
data class SecurityAssignment(
    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @ProblemFactCollectionProperty
    val skills: Set<Skill>,

    @PlanningEntityCollectionProperty
    val meetings: List<Meeting>
) {
    @ai.timefold.solver.core.api.domain.solution.PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptySet(), emptyList())
}


In [7]:
import kotlin.random.Random

// Function to generate random locations around Geneva
fun generateRandomLocation(): Location {
    // Approximate latitude and longitude for Geneva
    val genevaLat = 46.2044
    val genevaLon = 6.1432
    val latOffset = Random.nextDouble(-0.05, 0.05) // ~5 km
    val lonOffset = Random.nextDouble(-0.05, 0.05)
    return Location(genevaLat + latOffset, genevaLon + lonOffset)
}

// Function to generate fake agents
fun generateAgents(
    numberOfAgents: Int,
    numberOfSkills: Int
): List<Agent> {
    val skills = Skill.values().take(numberOfSkills).toSet()
    return (1..numberOfAgents).map { i ->
        Agent(
            id = i.toLong(),
            name = "Agent $i",
            homeLocation = generateRandomLocation(),
            vehicleHomeLocation = generateRandomLocation(),
            skills = skills.shuffled().take(Random.nextInt(1, skills.size + 1)).toSet(),
            weeklyWorkloadHours = 40
        )
    }
}

// Function to generate fake meetings
fun generateMeetings(
    numberOfMeetings: Int,
    skills: Set<Skill>
): List<Meeting> {
    return (1..numberOfMeetings).map { i ->
        val requiredSkills = skills.shuffled().take(Random.nextInt(1, 3)).toSet()
        val isNight = Random.nextBoolean() && Random.nextDouble() > 0.7 // 30% chance
        val timeWindowStart = if (isNight) LocalTime.of(20, 0) else LocalTime.of(8, 0)
        val timeWindowEnd = if (isNight) LocalTime.of(23, 0) else LocalTime.of(18, 0)
        Meeting(
            id = i.toLong(),
            clientName = "Client $i",
            location = generateRandomLocation(),
            requiredSkills = requiredSkills,
            durationHours = Random.nextInt(1, 3),
            timeWindowStart = timeWindowStart,
            timeWindowEnd = timeWindowEnd,
            isNightMeeting = isNight
        )
    }
}

// Parameters for data generation
val numberOfAgents = 5
val numberOfSkills = 3
val numberOfMeetings = 15

// Generate skills
val allSkills = Skill.values().take(numberOfSkills).toSet()

// Generate agents and meetings
val agents = generateAgents(numberOfAgents, numberOfSkills)
val meetings = generateMeetings(numberOfMeetings, allSkills)

// Create the solution
val problem = SecurityAssignment(
    agents = agents,
    skills = allSkills,
    meetings = meetings
)


In [9]:
// Place the revised SecurityConstraintProvider class here
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.LocalTime

class SecurityConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            // Hard Constraints
            agentSkillConstraint(constraintFactory),
            timeWindowConstraint(constraintFactory),
            weeklyWorkloadConstraint(constraintFactory),
            // Soft Constraints
            minimizeNumberOfAgentsConstraint(constraintFactory),
            minimizeNightMeetingsOverlapConstraint(constraintFactory)
        )
    }

    /**
     * Hard Constraint:
     * Ensure that the assigned agent has all the required skills for the meeting.
     */
    fun agentSkillConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                val agent = meeting.assignedAgent
                agent != null && !meeting.requiredSkills.all { it in agent.skills }
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent lacks required skills")
    }

    /**
     * Hard Constraint:
     * Ensure that the meeting is scheduled within the agent's available time window.
     */
    fun timeWindowConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                val agent = meeting.assignedAgent
                agent != null && !isWithinTimeWindow(meeting, agent)
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Meeting outside agent's available time window")
    }

    /**
     * Helper function to check if a meeting is within the agent's available hours.
     */
    fun isWithinTimeWindow(meeting: Meeting, agent: Agent): Boolean {
        val availableHours = agentAvailableHours(agent)
        return availableHours.contains(meeting.timeWindowStart) && availableHours.contains(meeting.timeWindowEnd)
    }

    /**
     * Define the agent's available hours.
     * For simplicity, agents are available from 8 AM to 6 PM and 8 PM to 11 PM.
     */
    fun agentAvailableHours(agent: Agent): List<LocalTime> {
        val dayHours = (8..18).map { LocalTime.of(it, 0) }
        val nightHours = (20..23).map { LocalTime.of(it, 0) }
        return dayHours + nightHours
    }

    /**
     * Hard Constraint:
     * Ensure that agents do not exceed their weekly workload (e.g., 40 hours).
     */
    fun weeklyWorkloadConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Agent::class.java)
            .join(Meeting::class.java,
                Joiners.equal { agent: Agent -> agent },
                Joiners.filter { agent, meeting ->
                    meeting.assignedAgent == agent
                }
            )
            .groupBy({ _, _ -> }, ConstraintCollectors.sum(Meeting::durationHours))
            .filter { totalHours -> totalHours > 40 }
            .penalize(HardSoftScore.ONE_HARD, { _, excess -> excess })
            .asConstraint("Agent exceeds weekly workload")
    }

    /**
     * Soft Constraint:
     * Prefer to use fewer agents by minimizing the number of agents assigned to meetings.
     */
    fun minimizeNumberOfAgentsConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Agent::class.java)
            .join(Meeting::class.java,
                Joiners.equal { agent: Agent -> agent },
                Joiners.notNull(Meeting::assignedAgent)
            )
            .groupBy({ agent, _ -> agent }, ConstraintCollectors.count())
            .penalize(HardSoftScore.ONE_SOFT, { _, _ -> 1 })
            .asConstraint("Prefer fewer agents")
    }

    /**
     * Soft Constraint:
     * Minimize overlapping night meetings for the same agent.
     */
    fun minimizeNightMeetingsOverlapConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEachUniquePair(Meeting::class.java,
            Joiners.equal(Meeting::assignedAgent),
            Joiners.equal(Meeting::isNightMeeting)
        )
            .penalize(HardSoftScore.ONE_SOFT)
            .asConstraint("Minimize overlapping night meetings")
    }
}


Line_9.jupyter.kts (64:29 - 34) Parameter 'agent' is never used
Line_9.jupyter.kts (76:14 - 18) None of the following functions can be called with the arguments supplied: 
public open fun <B : Any!> join(p0: UniConstraintStream<TypeVariable(B)!>!, p1: BiJoiner<Line_6_jupyter.Agent!, TypeVariable(B)!>!, p2: BiJoiner<Line_6_jupyter.Agent!, TypeVariable(B)!>!): BiConstraintStream<Line_6_jupyter.Agent!, TypeVariable(B)!>! defined in ai.timefold.solver.core.api.score.stream.uni.UniConstraintStream
public abstract fun <B : Any!> join(p0: UniConstraintStream<TypeVariable(B)!>!, vararg p1: BiJoiner<Line_6_jupyter.Agent!, TypeVariable(B)!>!): BiConstraintStream<Line_6_jupyter.Agent!, TypeVariable(B)!>! defined in ai.timefold.solver.core.api.score.stream.uni.UniConstraintStream
public open fun <B : Any!> join(p0: Class<TypeVariable(B)!>!, p1: BiJoiner<Line_6_jupyter.Agent!, TypeVariable(B)!>!, p2: BiJoiner<Line_6_jupyter.Agent!, TypeVariable(B)!>!): BiConstraintStream<Line_6_jupyter.Agent!, Type

In [ ]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import java.time.Duration

// Configure the solver
val solverFactory: SolverFactory<SecurityAssignment> = SolverFactory.create(SolverConfig()
    .withSolutionClass(SecurityAssignment::class.java)
    .withEntityClasses(Meeting::class.java)
    .withConstraintProviderClass(SecurityConstraintProvider::class.java)
    // Terminate after 10 seconds for this example
    .withTerminationSpentLimit(Duration.ofSeconds(10))
)

// Build the solver
val solver: Solver<SecurityAssignment> = solverFactory.buildSolver()

// Solve the problem
println("Solving the problem...")
val solution: SecurityAssignment = solver.solve(problem)
println("Solving finished with score: ${solution.score}")


In [ ]:
// Function to display assignments
fun displayAssignments(solution: SecurityAssignment) {
    println("\nSecurity Assignments:")
    println("---------------------")
    solution.meetings.forEach { meeting ->
        val agent = meeting.assignedAgent
        if (agent != null) {
            println("Meeting '${meeting.clientName}' assigned to ${agent.name} at location (${agent.location.latitude}, ${agent.location.longitude})")
        } else {
            println("Meeting '${meeting.clientName}' is unassigned.")
        }
    }
}

// Display the assignments
displayAssignments(solution)
